In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import sys
import pickle
from collections import defaultdict
import json
import numpy as np
import math
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB
import time
import bz2
import zlib

## lAST yEAR
# from word_definite import *
# from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
import winsound
def playBeep():
    for i in range(3):
        winsound.Beep(2200, 300)
        winsound.Beep(2600, 300)

In [4]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [5]:
dataset_4k_1k = pickle.load(open('../SmallDataset_4K_1K.p', 'rb'))
TrainFiles = dataset_4k_1k['TrainFiles']
TestFiles = dataset_4k_1k['TestFiles']

dataset_6k_3k = pickle.load(open('../SmallDataset_6K_3K.p', 'rb'))
TrainFiles_2 = dataset_6k_3k['TrainFiles']
TestFiles_2 = dataset_6k_3k['TestFiles']

In [6]:
matDB = MatDB.MatDB()

In [7]:
# from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)

In [8]:
"""
################################################################################################
##############################  GET A FILENAME TO SAVE WEIGHTS  ################################
################################################################################################
"""
import time
st = str(int((time.time() * 1e6) % 1e13))
log_name = 'logs/train_nnet_t{}.out'.format(st)
p_name = 'outputs/train_nnet_t{}.p'.format(st)
print('nEURAL nET wILL bE sAVED hERE: ', p_name)

nEURAL nET wILL bE sAVED hERE:  outputs/train_nnet_t508288462587.p


In [9]:
trainingStatus = defaultdict(lambda: bool(False))

In [33]:
class Trainer:
    def __init__(self):
        self.hidden_layer_size = 300
        self._edge_vector_dim = WD._edge_vector_dim
        # self._full_cnglist = list(WD.mat_cngCount_1D)
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
        self.history = defaultdict(lambda: list())

    def SaveToMem(self, sentenceObj, dcsObj, _debug = True):
        
        """ Pre-Process DCS and SKT to get all Nodes etc. """
        try:
            (nodelist, nodelist_correct, nodelist_to_correct_mapping) = GetTrainingKit(sentenceObj, dcsObj)
        except IndexError as e:
            # print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            # print(e)
            return
        
#         startT = time.time()
        """ SKT FEATURE VECTOR MATRIX """
        conflicts_Dict_correct = Get_Conflicts(nodelist_correct)
        featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)
        
        """ SKT FEATURE VECTOR MATRIX """
        conflicts_Dict = Get_Conflicts(nodelist)
        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
#         print('Nodelen: {}, Time taken to create: {}'.format(len(nodelist), time.time() - startT))
        
        with bz2.BZ2File('../NewData/skt_dcs_DS.bz2/' + sentenceObj.sent_id + '.ds.bz2', 'w') as f:
            pickle.dump({
                    'nodelist': nodelist,
                    'nodelist_correct': nodelist_correct,
                    'nodelist_to_correct_mapping': nodelist_to_correct_mapping,
                    'conflicts_Dict_correct': conflicts_Dict_correct,
                    'featVMat_correct': featVMat_correct,
                    'conflicts_Dict': conflicts_Dict,
                    'featVMat': featVMat
                }, f)
        
#         with open('../NewData/skt_dcs_DS/' + sentenceObj.sent_id + '.ds', 'wb') as f:
#             pickle.dump({
#                     'nodelist': nodelist,
#                     'nodelist_correct': nodelist_correct,
#                     'nodelist_to_correct_mapping': nodelist_to_correct_mapping,
#                     'conflicts_Dict_correct': conflicts_Dict_correct,
#                     'featVMat_correct': featVMat_correct,
#                     'conflicts_Dict': conflicts_Dict,
#                     'featVMat': featVMat
#                 }, f)
        
            
#         startT = time.time()
#         combined_loader = pickle.load(open('../NewData/skt_dcs_DS/' + sentenceObj.sent_id + '.ds', 'rb'))
#         print('Time taken to load from pickle file: {}'.format(time.time() - startT))
        
#         startT = time.time()
#         combined_loader = pickle.load(bz2.BZ2File('../NewData/skt_dcs_DS.bz2/' + sentenceObj.sent_id + '.ds.bz2', 'r'))
#         print('Time taken to load from bz2 file: {}\n'.format(time.time() - startT))

In [25]:
# trainer.neuralnet.U

In [34]:
trainer = None
def InitModule(_matDB):
    global WD, trainer
    WD.word_definite_extInit(_matDB)
    trainer = Trainer()
InitModule(matDB)
trainingStatus = defaultdict(lambda: bool(False))
# trainer.Load('outputs/train_nnet_t427031523027.p')

In [35]:
"""
################################################################################################
##############################  TRAIN FUNCTION  ################################################
################################################################################################
"""

def save_all_bz2(loaded_SKT, loaded_DCS, n_checkpt = 100):
    file_counter = 0

    for fn in loaded_DCS.keys():
        if file_counter % n_checkpt == 0:
            print(file_counter,' Checkpoint... ')
            sys.stdout.flush() # Flush IO buffer 
        _ = trainer.SaveToMem(loaded_SKT[fn], loaded_DCS[fn])
        file_counter += 1


In [36]:
save_all_bz2(loaded_SKT, loaded_DCS, n_checkpt=50)

0  Checkpoint... 


KeyboardInterrupt: 

In [26]:
for fn in TestFiles[:10]:
    _ = trainer.Test(loaded_SKT[fn], loaded_DCS[fn])

Get Edge Features Time:  6.570472478866577
Get Edge Features Time:  1.6002552509307861
Get Edge Features Time:  0.5128428936004639
Get Edge Features Time:  0.16260337829589844
Get Edge Features Time:  0.30678248405456543
Get Edge Features Time:  1.3092880249023438
Get Edge Features Time:  0.8916211128234863
Get Edge Features Time:  1.8754069805145264
Get Edge Features Time:  1.14072585105896
Get Edge Features Time:  0.27155017852783203


In [15]:
WD.word_definite_extInit(matDB)
# node2 = WD.word_definite('tIkzRam', 'tIkzRa', 31, 0, 0)
node1 = WD.word_definite('Adi', 'Adi', 31, 0, 0)
node2 = WD.word_definite('ca', 'ca', 2, 0, 1)
# node1 = WD.word_definite('koRam', 'koRa', 31, 0, 1)
print(node1)
print(node2)

feats = WD.Get_Features(node1, node2)
print(feats.shape)

WD_Node[C: 0, P: 0, Adi @(31) => Adi]
WD_Node[C: 1, P: 0, ca @(2) => ca]
(1000, 1)


In [17]:
feats.nbytes*900/(1024)

7031.25

### Implement Pooled Test Function

In [ ]:
import Train_n_Save_NNet
import multiprocessing as mp

In [41]:
def test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = None, n_checkpt = 100):
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    total_output_nodes = 0
    correct_word = 0;
    file_counter = 0
    if _testFiles is None:
        if n_testSet == -1:
            _testFiles = TestFiles
        else:
            _testFiles = TestFiles[0:n_testSet]
    else:
        if n_testSet == -1:
            _testFiles = _testFiles
        else:
            _testFiles = _testFiles[0:n_testSet]
            
    recalls = []
    recalls_of_word = []
    precisions = []
    precisions_of_words = []
    for fn in _testFiles:
        if file_counter % n_checkpt == 0:
            print(file_counter,' Checkpoint... ')
            sys.stdout.flush() # Flush IO buffer 
        file_counter += 1
        sentenceObj = loaded_SKT[fn]
        dcsObj = loaded_DCS[fn]        
        try:
            (word_match, lemma_match, n_dcsWords, n_output_nodes) = trainer.Test(sentenceObj, dcsObj)
            
            recalls.append(lemma_match/n_dcsWords)
            recalls_of_word.append(word_match/n_dcsWords)
            
            precisions.append(lemma_match/n_output_nodes)
            precisions_of_words.append(word_match/n_output_nodes)
            
            total_lemma += n_dcsWords
            total_word += n_dcsWords
            
            total_output_nodes += n_output_nodes            
            
            correct_lemma += lemma_match
            correct_word += word_match
        except (IndexError, KeyError) as e:
            print('Failed!')        

    print('Avg. Micro Recall of Lemmas: {}'.format(np.mean(np.array(recalls))))
    print('Avg. Micro Recall of Words: {}'.format(np.mean(np.array(recalls_of_word))))
    print('Avg. Micro Precision of Lemmas: {}'.format(np.mean(np.array(precisions))))
    print('Avg. Micro Precision of Words: {}'.format(np.mean(np.array(precisions_of_words))))
    
    return (recalls, recalls_of_word, precisions, precisions_of_words)

In [ ]:
_ = test(loaded_SKT, loaded_DCS, n_testSet = 3000, _testFiles = TestFiles_2, n_checkpt = 100)

8

######   THE END
-----------------------------------------------------------------------

## Multiprocessing Testing

In [8]:
from multiprocessing import Process
import os
import poolTest
import sys

In [17]:
from multiprocessing import Pool
import pickle

In [18]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')

In [11]:
# USING PROCESS OBJECT
glob_arr = list(range(100))

# if __name__ == '__main__':
proc_count = 4

procs = [None]*proc_count

for i in range(proc_count):
    procs[i] = Process(target = poolTest.f, args = (i, ))
for proc in procs:
    proc.start()
for proc in procs:
    proc.join()
    print('Done: ', proc.pid)

Done:  9608
Done:  5124
Done:  13260
Done:  7332


In [21]:
sktObjList = list(loaded_SKT.values())

In [26]:
# USING POOL
p = Pool(4)
chunk_counts = p.map(poolTest.counter, sktObjList[:20])
p.close()

chunk_counts

[('14741', 8),
 ('21171', 9),
 ('44699', 5),
 ('286767', 3),
 ('268297', 3),
 ('253892', 1),
 ('42110', 5),
 ('138854', 3),
 ('42273', 6),
 ('320783', 4),
 ('273356', 5),
 ('33423', 9),
 ('40411', 6),
 ('343323', 4),
 ('249124', 5),
 ('249672', 6),
 ('214118', 6),
 ('39926', 1),
 ('437631', 2),
 ('289985', 5)]